<a href="https://colab.research.google.com/github/TomiOdrio/RatingPrediction/blob/main/Aplicaci%C3%B3nModeloFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Sube un archivo de Kaggle a la Máquina Virtual de Colab en un .zip
#Requiere tener el archivo 'kaggle.json' en Drive
#Requiere aceptar las reglas de la competencia en Kaggle

import json
from google.colab import drive

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

drive.mount('/content/drive', force_remount=True)
with open("/content/drive/My Drive/kaggle.json", 'r') as f:
    api_token= json.load(f)

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c tp2-predictivo-2023q2

Mounted at /content/drive
 62% 25.0M/40.2M [00:00<00:00, 115MB/s] 
100% 40.2M/40.2M [00:00<00:00, 139MB/s]


In [ ]:
#Extraigo todos los archivos que están en el .zip que acabamos de "bajar".
import zipfile
import os

os.listdir()


for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

In [ ]:
_=!pip install feature_engine

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#exportar pipes
import pickle
import joblib

#crear directorio caché
from tempfile import mkdtemp
from shutil import rmtree

# from tqdm import tqdm
# from sklearn import set_config
# from sklearn.base import clone

#pipeline y search
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, ParameterGrid, train_test_split, cross_val_score, KFold
from sklearn.feature_selection import SelectPercentile, chi2

#modelos
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.impute import SimpleImputer

#transformaciones
from feature_engine.encoding import RareLabelEncoder, OrdinalEncoder, OneHotEncoder, CountFrequencyEncoder
from feature_engine.imputation import AddMissingIndicator, MeanMedianImputer, CategoricalImputer, EndTailImputer, RandomSampleImputer
from feature_engine.transformation import PowerTransformer
from feature_engine.creation import RelativeFeatures
from feature_engine.wrappers import SklearnTransformerWrapper

#distribuciones
import scipy as sp
from scipy.stats import randint, uniform

# from sklearn.compose import ColumnTransformer, make_column_transformer
# from sklearn.impute import KNNImputer, SimpleImputer
# from sklearn.preprocessing import StandardScaler, OneHotEncoder, KBinsDiscretizer

In [ ]:
test = pd.read_csv('/content/testear.csv')
test = test.drop(columns='Unnamed: 0')

#Correcciones Test
test['video'] = test['video'].replace(False,0)
test['video'] = test['video'].replace(True,1)
test['adult'] = test['adult'].replace(False,0)
test['adult'] = test['adult'].replace(True,1)

In [ ]:
test.loc[test['startYear'] == 0, 'startYear'] = float("NaN")
test.loc[test['endYear'] == 0, 'endYear'] = float("NaN")
test.loc[test['runtimeMinutes'] == 0, 'runtimeMinutes'] = float("NaN")
test.loc[test['language'] == '0', 'language'] = float("NaN")
test.loc[test['attributes'] == '0', 'attributes'] = float("NaN")
test.loc[test['budget'] == 0, 'budget'] = float("NaN")
test.loc[test['genres_x'] == '0', 'genres_x'] = float("NaN")
test.loc[test['genres_y'] == '0', 'genres_y'] = float("NaN")
test.loc[test['production_companies'] == '0', 'production_companies'] = float("NaN")
test.loc[test['production_countries'] == '0', 'production_countries'] = float("NaN")
test.loc[test['revenue'] == 0, 'revenue'] = float("NaN")
test.loc[test['runtime'] == 0, 'runtime'] = float("NaN")
test.loc[test['directors'] == '0', 'directors'] = float("NaN")
test.loc[test['writers'] == '0', 'writers'] = float("NaN")

In [ ]:
test_sub = test.drop(columns=['tagline','production_companies','production_countries'])

test_sub['writers'] = test_sub['writers'].replace(np.nan,'unk')
test_sub['cant_writers'] = test_sub['writers'].apply(lambda x: x.count(',')+1 if 'unk' not in x else np.nan)
test_sub['directors'] = test_sub['directors'].replace(np.nan,'unk')
test_sub['cant_directors'] = test_sub['directors'].apply(lambda x: x.count(',')+1 if 'unk' not in x else np.nan)
test_sub['genres_x'] = test_sub['genres_x'].replace(np.nan,'unk')
test_sub['cant_genres_x'] = test_sub['genres_x'].apply(lambda x: x.count(',')+1 if 'unk' not in x else np.nan)

txt = ['tagline'] if 'tagline' in test_sub.columns else []
bool = [c for c in test_sub.columns if test_sub[c].nunique() <= 2]
cat = [c for c in test_sub.columns if test_sub[c].dtype == 'O' and c not in bool]
num = [c for c in test_sub.columns if test_sub[c].dtype != 'O' and c not in bool]

print(f"Texto: {txt} \n Booleanas: {bool} \n Categóricas: {cat} \n Numéricas: {num} \n")
print("Total variables:",len(txt)+len(bool)+len(cat)+len(num))


Texto: [] 
 Booleanas: ['isAdult', 'isOriginalTitle', 'adult', 'video'] 
 Categóricas: ['titleType', 'genres_x', 'directors', 'writers', 'language', 'attributes', 'genres_y', 'original_language', 'status'] 
 Numéricas: ['numVotes', 'startYear', 'endYear', 'runtimeMinutes', 'seasonNumber', 'episodeNumber', 'ordering', 'budget', 'popularity', 'revenue', 'runtime', 'cant_writers', 'cant_directors', 'cant_genres_x'] 

Total variables: 27


In [ ]:
pipe = joblib.load('/content/drive/My Drive/notebooks/TP2_Predictivo/pipe.joblib')
modelo = joblib.load('/content/drive/My Drive/notebooks/TP2_Predictivo/modelo.joblib')

In [ ]:
# Transformación con Pipeline
test_sub = pipe.transform(test_sub)
for var in cat:
  test_sub[var] = test_sub[var].replace(np.nan,0)

# Predicción con Modelo
y_pred = modelo.predict(test_sub)

# Exporto los resultados
salida = pd.DataFrame(data={"averageRating": y_pred}).astype(str)
salida.index = test_sub.index
salida.to_csv("modelo.csv", sep=',',index=True,  index_label='Id')

/usr/local/lib/python3.10/dist-packages/feature_engine/encoding/base_encoder.py:255: UserWarning: During the encoding, NaN values were introduced in the feature(s) genres_x, directors, writers, attributes, genres_y, original_language, status.
  warnings.warn(
